Author: Kareem Hamoudeh

1. Look at the [chapter on interactive graphics](https://smart-stats.github.io/ds4bio_book/book/_build/html/interactive.html) and, specifically, the code to display a subject's MRICloud data as a sunburst plot. Do the following. Display this subject's data as a [Sankey diagram](https://plotly.com/python/sankey-diagram/). Display as many levels as you can (at least 3) for Type = 1, starting from the intracranial volume. Put this in a file called hw4.ipynb.

In [12]:
import pandas as pd
import plotly.express as px
import numpy as np

In [13]:
dat = pd.read_csv("assetts/kirby21.csv").drop(['Unnamed: 0'], axis = 1)
dat = dat.assign(id_char = dat.id.astype(str))
icv = dat.groupby(['id']).volume.sum().reset_index().rename(columns = {'volume' : 'icv'})
dat = pd.merge(dat, icv, on = 'id')
dat = dat.assign(comp = dat.volume / dat.icv)
## load in the hierarchy information
url = "https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt"
multilevel_lookup = pd.read_csv(url, sep = "\t").drop(['Level5'], axis = 1)
multilevel_lookup = multilevel_lookup.rename(columns = {
    "modify"   : "roi", 
    "modify.1" : "level4",
    "modify.2" : "level3", 
    "modify.3" : "level2",
    "modify.4" : "level1"})
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]
## Now load in the subject data
id = 127
subjectData = pd.read_csv("assetts/kirby21AllLevels.csv")
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == id)]
subjectData = subjectData[['roi', 'volume']]
## Merge the subject data with the multilevel data
subjectData = pd.merge(subjectData, multilevel_lookup, on = "roi")
subjectData = subjectData.assign(icv = "ICV")
subjectData = subjectData.assign(comp = subjectData.volume / np.sum(subjectData.volume))

## Sankey Plot 

1) Make tables for connections. ICV-> 1, 1->2, 2>3 etc...
2) Label list
3) for loop for each pair  (ICV - CSF/Diencephelon... | Level1 - CSF/Die...)
4) populate source/target/value with numbers corresponding to the indices of list of labels


labels = brain structures 

value = composition

source = for level0-1: ICV| level1-2: level1 | Level2-3: level2| level3-4

target = 


In [14]:
# Grouping level 0 (icv) and level 1
icv_one = subjectData.groupby(['icv', 'level1']).sum().reset_index().drop('volume', axis = 1)
one_two = subjectData.groupby(['level1', 'level2']).sum().reset_index().drop('volume', axis = 1)
two_three = subjectData.groupby(['level2', 'level3']).sum().reset_index().drop('volume', axis = 1)
three_four = subjectData.groupby(['level3', 'level4']).sum().reset_index().drop('volume', axis = 1)

In [15]:
import plotly.graph_objects as go


labels = ['ICV']
for i in icv_one['level1']:
    labels.append(i)
for i in one_two['level2']:
    labels.append(i)
for i in two_three['level3']:
    labels.append(i)
for i in three_four['level4']:
    labels.append(i)


value = []

for i in icv_one['comp']:
    value.append(i)
for i in one_two['comp']:
    value.append(i)
for i in two_three['comp']:
    value.append(i)
for i in three_four['comp']:
    value.append(i)




## Sankey of level0(icv) to level1

In [16]:
# Trying to make a for loop to append source and target lists 
index_list = []
for i in range(226):
    index_list.append(i)
sourcetest = []
for i in icv_one['icv']:
    level1_index = labels.index(i)
    sourcetest.append(index_list[level1_index])

for i in one_two['level1']:
    level1_index = labels.index(i)
    sourcetest.append(index_list[level1_index])

# for i in two_three['level2']:
#     level1_index = labels.index(i)
#     sourcetest.append(index_list[level1_index])

# for i in three_four['level3']:
#     level1_index = labels.index(i)
#     sourcetest.append(index_list[level1_index])

targettest = []
for i in icv_one['level1']:
    level2_index = labels.index(i)
    targettest.append(index_list[level2_index])

for i in one_two['level2']:
    level2_index = labels.index(i)
    targettest.append(index_list[level2_index])

# for i in two_three['level3']:
#     level2_index = labels.index(i)
#     targettest.append(index_list[level2_index])

# for i in three_four['level4']:
#     level2_index = labels.index(i)
#     targettest.append(index_list[level2_index])




### Displaying for 3 levels 

**ICV -> level1 -> level2

In [17]:
# data to dict, dict to sankey
link = dict(source = sourcetest, target = targettest, value = value)
node = dict(label = labels, pad=15, thickness=5)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.show()
fig.write_html('graph0-2.html')

In [18]:


for i in two_three['level2']:
    level1_index = labels.index(i)
    sourcetest.append(index_list[level1_index])



for i in two_three['level3']:
    level2_index = labels.index(i)
    targettest.append(index_list[level2_index])


# data to dict, dict to sankey
link = dict(source = sourcetest, target = targettest, value = value)
node = dict(label = labels, pad=2, thickness=5)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.show()
fig.write_html('graph0-3.html')

In [19]:
for i in three_four['level3']:
    level1_index = labels.index(i)
    sourcetest.append(index_list[level1_index])

for i in three_four['level4']:
    level2_index = labels.index(i)
    targettest.append(index_list[level2_index])

# data to dict, dict to sankey
link = dict(source = sourcetest, target = targettest, value = value)
node = dict(label = labels, pad=2, thickness=5)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.show()
fig.write_html('graph0-4')

Create the opioid sqlite database from https://smart-stats.github.io/ds4bio_book/book/_build/html/sqlite.html. However, only go to the step where the csv files are read into the database. Then exit sqlite and you should have a file opioid.db that has the data. Next, read the three tables into pandas dataframes and do the data wrangling from the sqlite chapter directly in pandas. Add the python code to your hw4.ipynb file.


Create an interactive scatter plot of average number of opiod pills by year plot using plotly. See the example here. Don't do the intervals (little vertical lines), only the points. Add your plot to an html file with your repo for your Sanky diagram and host it publicly. Put a link to your hosted file in a markdown cell of your hw4.ipynb file. Note, an easy way to create a webpage with this graphic is to export an ipynb as an html file.

In [71]:
# Reading the data into pandas df

import sqlite3
import plotly.express as px

connection = sqlite3.connect('opioid.db')

In [72]:
opiods_annual = pd.read_sql("SELECT * from annual", connection)
opioids_land = pd.read_sql("SELECT * from land", connection)
opioids_population = pd.read_sql("SELECT * from population", connection)


In [73]:
connection.close()

In [74]:
annual = opiods_annual.dropna()
land = opioids_land.dropna()
population = opioids_population.dropna()

In [75]:
annual["DOSAGE_UNIT"] = pd.to_numeric(annual["DOSAGE_UNIT"], errors="coerce").astype("float")


In [76]:
annual["Pills_in_millions"] = annual["DOSAGE_UNIT"] / 1000000


In [80]:
annual_grouped = annual.groupby("year")["Pills_in_millions"].mean().reset_index()

In [81]:
annual_grouped

,year,Pills_in_millions
0,2006,2.645410
1,2007,2.992099
2,2008,3.252162
3,2009,3.526038
4,2010,3.783656
5,2011,4.035583
6,2012,3.993064
7,2013,3.861752
8,2014,3.768738


In [82]:
fig = px.scatter(annual_grouped, x="year", y="Pills_in_millions")

fig.update_layout(
    title="Average Number of Opioid Pills by Year",
    xaxis_title="Year",
    yaxis_title="Average Dosage Units",
)

fig.show()